In [2]:
import pandas as pd
import os
import numpy as np
import pandas_profiling

# This line is needed to display plots inline in Jupyter Notebook
%matplotlib inline

# Required for basic python plotting functionality
import matplotlib.pyplot as plt

# Required for formatting dates later in the case
import datetime
import matplotlib.dates as mdates

# Required to display image inline
from IPython.display import Image

# Advanced plotting functionality with seaborn
import seaborn as sns
sns.set(style="whitegrid") # can set style depending on how you'd like it to look

import folium  #needed for interactive map
from folium.plugins import HeatMap

import geopandas
from shapely.geometry import Polygon
from shapely.geometry import Point
from geopandas import GeoDataFrame

In [7]:
# Opens the map as a GeoDataFrame
path = 'Dataset/Original'
nyc_bouroughs = geopandas.read_file(path + '/Neighborhood Tabulation Areas.geojson')

path = 'Dataset/Parquets'
yellow_trips_2 = pd.read_parquet(path + '/mta_trips_filtered.parquet', engine = 'pyarrow')

In [8]:
geometry = [Point(xy) for xy in zip(mta_trips.longitude, mta_trips.latitude)]
# df = df.drop(['Lon', 'Lat'], axis=1)
crs = {'init': 'epsg:4326'}
gdf = GeoDataFrame(mta_trips, crs=crs, geometry=geometry)

In [9]:
print(gdf.shape)
gdf_boroughs = geopandas.sjoin(gdf, nyc_bouroughs, how="inner", op='intersects')
print(gdf.shape)

(7554197, 11)
(7554197, 11)


In [10]:
gdf_final = gdf_boroughs.drop(['geometry'], axis = 1)

In [11]:
gdf_final['line_name'] = gdf_final['line_name'].astype('str')
gdf_final.dtypes

station                object
line_name              object
division               object
audit_type             object
unit_id                object
datetime       datetime64[ns]
new_entries             int64
new_exits               int64
latitude              float64
longitude             float64
index_right             int64
ntacode                object
shape_area             object
county_fips            object
ntaname                object
shape_leng             object
boro_name              object
boro_code              object
dtype: object

In [12]:
path = 'Dataset/Parquets'
gdf_final.to_parquet(path + '/mta_trips_nta.parquet', engine = 'pyarrow')